(vis-matplotlib)=
# Highly Customisable Data Visualisation with Matplotlib


## Introduction

Here you'll see how to make just about any plot you can possibly imagine using the ferociously powerful *imperative* graphing package, **matplotlib**. If you read on to the chapter on {ref}`vis-narrative`, you'll see just how flexible it is. **matplotlib** is behind quite a few other data visualisation libraries, such as **plotlnine** and **seaborn**, which shows how much it can do!

For a more in-depth introduction to matplotlib, head over to [this tutorial](https://github.com/rougier/matplotlib-tutorial). This chapter is indebted to that tutorial and to the excellent [**matplotlib** documentation](https://matplotlib.org/stable/tutorials/index.html).

As ever, we'll start by importing some key packages:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Set seed for random numbers
seed_for_prng = 78557
prng = np.random.default_rng(seed_for_prng)  # prng=probabilistic random number generator

In [2]:
import matplotlib_inline.backend_inline

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

# Set max rows displayed for readability
pd.set_option("display.max_rows", 6)

## Understanding Matplotlib

You'll see we imported `matplotlib.pyplot as plt` above; this is the main part of **matplotlib** that we'll use in practice.

### The two **matplotlib** APIs

There are actually two ways to use **matplotlib**: the 'pyplot API' and the 'object-oriented API'. The pyplot API (application programming interface) gives an experience that's much closer to the paid programming language Matlab and can be useful to make a quick chart. This is no accident; **matplotlib** originally grew out of a desire to provide plotting functionality that was similar to what was found in Matlab. However, it has subsequently taken on its own style and API, and these developments are now much more powerful and convenient for plotting in Python.

The Matlab-style "pyplot-API" is very simple to use, for example:

In [ ]:
plt.plot([1, 2, 3, 4], [1, 4, 9, 16])

but this simplicity comes with costs down the line. So **we highly recommend that you use the "object oriented API"**, which the rest of this chapter will show you how to use.

```{admonition} Tip
:class: tip
**Matplotlib** returns an object when used, eg `[<matplotlib.lines.Line2D...` above. To suppress this, end the command with a semi-colon, `;`.
```

However, the pyplot API is generally less-flexible and less-useful than the object-oriented API, and for the rest of this chapter we'll be using the object-oriented API.

The object-oriented API is most often used by creating two objects: the figure and the axes. You should think of the figure object, `fig`, as the canvas on which you can put any number of charts. Each ax (short for 'axis') object is one chart within a figure. Of course, most of the time you're likely only to have one axis per figure, but in the cases when you don't it's a really useful setup. The plotting of elements such as lines, points, bars, and so, are controlled by the `ax` objects while the overall settings are controlled by `fig`.

Let's see an example of a line chart using this object-oriented approach.

In [ ]:
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.scatter([1, 2, 3, 4, 5, 6], [1, 4, 2, 3, 1, 7], s=150, c="b")
# Plot some data on the axes.

In the above example, we used `ax.scatter` to get a scatter plot, `s=150` to set the area of the points, and `c='b'` to set the color. Many of these features will accept an array instead of a single value and will map them into the plot in the way you'd expect, for instance:

In [ ]:
fig, ax = plt.subplots()
ax.scatter(
    [1, 2, 3, 4, 5, 6],
    [1, 4, 2, 3, 1, 7],
    s=np.linspace(300, 2000, 6),
    c=["b", "r", "g", "k", "cyan", "yellow"],
    edgecolors="k",
    alpha=0.5,
);

Here we asked for a different colour for each point, an area that's increasing linearly, partly transparent points (default is `alpha=1`, which is a solid colour), and a black edge colour.

You can probably begin to see how everything is going to be customisable. We've only seen aspects of the plot that are customisable through the `scatter` keyword so far though; let's now see an example that's a bit more real (and useful!) in which we'll want to add labels, a title, and more. We'll use the Midwest demographics dataset.

In [ ]:
df = pd.read_csv(
    "https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/midwest.csv",
    index_col="PID",
).drop("Unnamed: 0", axis=1)
df.head()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df["area"], df["poptotal"], edgecolors="k", alpha=0.6)
ax.set_xlim(0, 0.1)
ax.set_ylim(0, 1e6)
ax.set_xlabel("Area")
ax.set_ylabel("Population")
ax.set_title("Area vs. Population", loc="right");

Perhaps you can see from this already that **matplotlib** offers a ton of customisation features *and* that it can be quite verbose.

Let's see another couple of customisations that are really useful: formatting axes. The y-axis has already been formatted because we used a specific style file at the top of the page (the line beginning `plt.style.use`) but, on the x-axis, we'll add a percentage suffix on the numbers plus some minor tick marks.

In [ ]:
from matplotlib.ticker import AutoMinorLocator

fig, ax = plt.subplots()
ax.scatter(df["area"], df["poptotal"], edgecolors="k", alpha=0.6)
ax.set_xlim(0, 0.1)
ax.set_ylim(0, 1e6)
ax.set_xlabel("Area")
ax.set_ylabel("Population")
ax.set_title("Area vs. Population", loc="right")
ax.xaxis.set_minor_locator(AutoMinorLocator(4))
ax.xaxis.set_major_formatter("{x:.2f}%")
ax.ticklabel_format(style="sci", scilimits=(-2, 2), axis="y", useMathText=True);

The `AutoMinorLocator(4)` inserts 4 minor tick marks between each major tick mark. The major formatter is `'{x:.2f}%'`, which says print 2 decimal places followed by a % sign. Finally, the `ax.ticklabel_format` option changes the computer science notation to use latex to print a prettier form of the one million signifier.

Rather than go through all of the many, many options for customisation, the figure below (from the Matplotlib documentation) gives an overview of the options:

![Anatomy of a matplotlib figure](https://matplotlib.org/_images/anatomy.png)

Okay a quick overview of what the most important plot elements are:

- Figure, or 'fig': the figure keeps track of all the child Axes that are on it and a smattering of 'special' artists (titles, figure legends, etc). A figure can contain any number of Axes, but will typically have at least one.
- Axes, or 'ax': this is the plot, the region of the image that traces out the data. A given Figure can contain many Axes, but a given Axes object can only be in one Figure. The Axes contains two (or three in the case of 3D) Axis objects (Axes and Axis are different things!) that record the data limits (you can override these via the `axes.Axes.set_xlim()` and `axes.Axes.set_ylim()` methods). Each Axes object has a title (set via `set_title()`), an x-label (set via `set_xlabel()`), and a y-label set via `set_ylabel()`). When you add, say, a line chart to an Axes object it appears as a Line2D object associated that that axis and it is created by calling a method on an Axes object.
- Axis: these are the number-line objects that control the limits of what the viewer can see. They also provide the means to access the ticks (the marks on the axis) and ticklabels (strings labeling the ticks). The location of the ticks is determined by a Locator object and the ticklabel strings are formatted by a Formatter. The combination of Locator and Formatter gives very fine control over the tick locations and labels.

You can find a detailed blog post on the ins and outs [here](https://dev.to/skotaro/artist-in-matplotlib---something-i-wanted-to-know-before-spending-tremendous-hours-on-googling-how-tos--31oo). The object-oriented structure of **matplotlib** plots can be confusing at first, but, as we'll see, it pretty much fades into the background when you're making plots.

While **matplotlib** is super-customisable, sometimes achieving what you want directly can be a bit verbose. Let's say we want to now differentiate these points with colour according to which state they belong to and add a legend that says which states have which colour. The easiest way to do this is by creating a for loop.

In [ ]:
colours = plt.get_cmap("Dark2")(np.linspace(0, 1, len(df["state"].unique())))

fig, ax = plt.subplots()
for i, state in enumerate(df["state"].unique()):
    xf = df.loc[df["state"] == state]
    ax.scatter(
        xf["area"],
        xf["poptotal"],
        color=colours[i],
        label=state,
        s=100,
        edgecolor="k",
        alpha=0.8,
    )
ax.set_xlim(0, 0.1)
ax.set_ylim(0, 1e6)
ax.set_xlabel("Area")
ax.set_ylabel("Population")
ax.set_title("Area vs. Population", loc="center")
ax.xaxis.set_minor_locator(AutoMinorLocator(4))
ax.xaxis.set_major_formatter("{x:.2f}%")
ax.ticklabel_format(style="sci", scilimits=(-2, 2), axis="y", useMathText=True)
ax.legend(title="State", loc="upper right");

Okay, so we managed to get what we wanted. We used a colormap to get 5 qualitatively different colours; there are also sequential colormaps for continuous (as opposed to discrete) variables. You can find out more about the colormaps available in base matplotlib [here](https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html). We also subsetted the dataframe and looped over it by state. Matplotlib doesn't always work so well with tidy data. As you saw in this example, we had to loop over the different states. Some of its defaults are a little more friendly when using data that is unstacked, eg with one state per column.

However, this was quite verbose; when we come to **seaborn**, we'll see a much easier way of doing this. 

For now, let's see a couple of other plot types that are useful, beginning with contour plots:

In [ ]:
def f(x, y):
    return np.sin(x) ** 10 + np.cos(10 + y * x) * np.cos(x)


x = np.linspace(0, 5, 100)
y = np.linspace(0, 5, 100)

X, Y = np.meshgrid(x, y)
Z = f(X, Y)

fig, ax = plt.subplots()
cf = ax.contourf(X, Y, Z, cmap="plasma")
ax.set_title(r"$f(x,y) = \sin^{10}(x) + \cos(x)\cos\left(10 + y\cdot x\right)$")
cbar = fig.colorbar(cf)

This demonstrates creating a heatmap (or contour plot) with a colour bar legend and a title that's rendered with latex. The heatmap uses a perceptually uniform distribution that makes equal changes look equal; **matplotlib** has a few of these. If you need more colours, check out the packages [**colorcet**](https://colorcet.holoviz.org/) and [**palettable**](https://jiffyclub.github.io/palettable/).

You can do some really quite amazing things using **matplotlib**'s build what you want philosophy. For instance, you can [build a timeline](https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/timeline.html#sphx-glr-gallery-lines-bars-and-markers-timeline-py), make [charts with polar axes](https://matplotlib.org/3.1.1/gallery/pie_and_polar_charts/polar_bar.html#sphx-glr-gallery-pie-and-polar-charts-polar-bar-py), and create [XKCD style plots](https://matplotlib.org/3.1.1/gallery/showcase/xkcd.html#sphx-glr-gallery-showcase-xkcd-py).

One basic bit of functionality that you might need is to put more than one type of information on a single plot. Using the object-oriented API, this is as simple as calling another method on an `ax` that you've already created. In the example below, we'll call `ax.hist` followed by `ax.plot` to get a the theoretical curve for a normal distribution (aka Gaussian) overlaid on a kernel density estimate based on many draws from the relevant distribution using `numpy`.

In [ ]:
rand_draws = np.random.randn(5000)
grid_x = np.linspace(-5, 5, 1000)

fig, ax = plt.subplots()
ax.hist(rand_draws, bins=50, density=True)
ax.plot(grid_x, 1 / np.sqrt(2 * np.pi) * np.exp(-(grid_x ** 2) / 2), linewidth=4);

Another important class of plots that we should look at are line charts, especially time series. Let's grab the real GDP time series for the UK and US and look at the year-on-year quarterly growth rates (the first few entries will be `NaN` because we switched to growth space).

In [ ]:
import pandas_datareader.data as web

ts_start_date = pd.to_datetime("1999-01-01")

df = pd.concat(
    [
        web.DataReader("ticker=RGDP" + x, "econdb", start=ts_start_date)
        for x in ["US", "UK"]
    ],
    axis=1,
)
df.columns = ["US", "UK"]
df.index.name = "Date"
df = 100 * df.pct_change(4)
df.head()

Okay, now the quick way to plot this would be to simply call `df.plot()`. That works fine and doesn't look too bad: it recognises that it's dealing with a datetime and plots sensible tick labels on the x-axis, and it produces a legend for each of the two time series. But let's say we want to make that plot quickly *and* do some fine-tuning with **matplotlib**: how can we? The answer is to create a `fig` and an `ax` and then to ask our dataframe to use the `ax` we already created to plot the data. We can then carry on using the `ax` object in any way we like.

In [ ]:
fig, ax = plt.subplots()
df.plot(ax=ax)
ax.set_title("Real GDP growth, %", loc="right")
ax.spines["right"].set_visible(True)
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.yaxis.tick_right()
# Put tick marks and tick labels on right-hand side

We could also have created this chart by looping through the different countries just as we looped through the difference states in the previous example.

Let's also see what happens when we use different limits:

In [ ]:
fig, ax = plt.subplots()
df.plot(ax=ax)
ax.set_title("Real GDP growth, %", loc="right")
ax.spines["right"].set_visible(True)
ax.spines["left"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.yaxis.tick_right()  # Put tick marks and tick labels on right-hand side
ax.set_xlim(pd.to_datetime("2018-01-01"), pd.to_datetime("2021-01-01"));

As you can see, the plot dynamically responded to the shorter time period by putting more details in, here of quarters. You can specify exactly what you want with the tick label formatters that cater to datetimes, but the defaults are pretty well-behaved.

Sometimes, you want to show different lines in different panels (or facets, as they are also known). You can do that with **matplotlib** too, though once again it's a build-your-own affair.

Let's put these two time series in their own facets to see how it works.

In [ ]:
fig, axes = plt.subplots(1, 2)
for i, ax in enumerate(axes):
    ax.plot(df.index, df.iloc[:, i])
    ax.set_title(df.columns[i], loc="left")
    ax.yaxis.tick_right()
    ax.spines["right"].set_visible(True)
    ax.spines["left"].set_visible(False)
    ax.spines["top"].set_visible(False)
    ax.set_ylim(df.min().min(), df.max().max())
fig.suptitle("Real GDP growth, %")
plt.tight_layout();

Quite a few things are happening here. The first is that we asked for more rows and columns from matplotlib using `plt.subplots(nrows, ncolumns)` and instead of returning a single `ax` it returned a list of axes (here of length 2). We then iterated over those axes using enumerate, which gives an integer `i` and an `ax` from the list, and plotted a column on each axis by subsetting the dataframe one column at a time using `df.iloc[:, i]`. Remember that the first position in `iloc` is for the index values (ie the rows), while the second is for the columns. Again, this was quite a lot of code to get a relatively simple chart done! And that's exactly why we're now going to take a look at the declarative library **seaborn** that wraps **matplotlib**.

### Seaborn

**seaborn** provides a high-level interface for quickly drawing standard charts. It is based on **matplotlib*, which is great because it means you can always tinker if you need to. And it plays very nicely with **pandas** dataframes too, so it can easily fit into your workflow.

Here's an overview of the API (application programming interface; basically the commands we can feed the library) for **seaborn**. (NB: this figure excludes the facet option, which combines multiple versions of the below plots.)

![seaborn api diagram](https://seaborn.pydata.org/_images/function_overview_8_0.png)

Let's see the chart we were just trying to make but rendered in **seaborn**. The first difference is that **seaborn** expects *tidy data*, a concept from the Data part of this book. So first we must transform our data into tidy format.



In [ ]:
tidy_df = df.stack().reset_index()
tidy_df.columns = ["Date", "Country", "Real GDP growth, %"]
tidy_df.head()

Okay now let's use seaborn to do the plotting. As with **matplotlib**, the way to get different charts in **seaborn** is a matter of knowing the API names. Here, we'll use `relplot`, which is for facets.

In [ ]:
import seaborn as sns

sns.relplot(
    data=tidy_df,
    x="Date",
    y="Real GDP growth, %",
    hue="Country",
    col="Country",
    kind="line",
    legend=False,
);

In this case, we got a very similar plot with less effort--and what we do is more generalisable too. As long as we have data in a tidy format, we can specify the columns to seaborn and let it put them in different facets. And facets aren't the only difference it can provide either: you can specify the hue such that it depends on another categorical variable (rather than, as in this case, changing with each facet). Here's an example changing the hue to reflect an extra type of information, region:


In [ ]:
# Download RGDP for Canada and France:
extra_df = pd.concat(
    [
        web.DataReader("ticker=RGDP" + x, "econdb", start=ts_start_date)
        for x in ["CA", "FR"]
    ],
    axis=1,
)
extra_df.columns = ["CA", "FR"]
extra_df.index.name = "Date"
extra_df = (100 * extra_df.pct_change(4)).stack().reset_index()
extra_df.columns = ["Date", "Country", "Real GDP growth, %"]
# Add the new data to the tidy dataframe
tidy_df = pd.concat([tidy_df, extra_df], axis=0)
continent_dict = {
    "CA": "North America",
    "US": "North America",
    "UK": "Europe",
    "FR": "Europe",
}
tidy_df["Region"] = tidy_df["Country"].map(continent_dict)
tidy_df.sample(6)

In [ ]:
sns.relplot(
    data=tidy_df,
    x="Date",
    y="Real GDP growth, %",
    hue="Region",
    col="Country",
    col_wrap=2,
    kind="line",
    legend=False,
);

As you can see, we now have two colours: one for each region.

Let's see some of the other useful shortcuts that **seaborn** provides over the top of **matplotlib**. First, correlation heatmaps:


In [ ]:
# Generate data and create corr mat
d = pd.DataFrame(
    data=np.random.normal(size=(100, 6)), columns=["A", "B", "C", "D", "E", "F"]
)
corr = d.corr()

# Generate a mask to cover the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Draw heatmap
sns.heatmap(
    corr,
    mask=mask,
    cmap="magma",
    vmin=-0.4,
    vmax=0.4,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
);

**seaborn** also offers convenience functions for (continuous) heatmaps, kernel density estimates, and marginal plots:


In [ ]:
df = sns.load_dataset("penguins")

g = sns.JointGrid(data=df, x="body_mass_g", y="bill_depth_mm", space=0)
g.plot_joint(
    sns.kdeplot,
    fill=True,
    clip=((2200, 6800), (10, 25)),
    thresh=0,
    levels=100,
    cmap="inferno",
)
g.plot_marginals(sns.histplot, color="#03051A", alpha=1, bins=25);

And plotting simple linear models according to a category:

In [ ]:
g = sns.lmplot(
    data=df,
    x="bill_length_mm",
    y="bill_depth_mm",
    hue="species",
    legend=False,
    line_kws={"lw": 1.0},
    height=5,
    aspect=1.75,
)
plt.legend(loc="lower left", frameon=True);

And, finally, violin plots:

In [ ]:
tips = sns.load_dataset("tips")
sns.violinplot(
    data=tips,
    x="day",
    y="total_bill",
    hue="smoker",
    split=True,
    inner="quart",
    linewidth=1,
);

### Plotnine

[**plotnine**](https://plotnine.readthedocs.io/en/stable/) is, like **seaborn**, a declarative library. Unlike **seaborn**, it adopts a 'grammar of graphics' approach inspired by the book 'The Grammar of Graphics' by Leland Wilkinson. **plotnine** is heavily inspired by the API of the popular **ggplot2** plotting package in the statistical programming language R. The point behind the grammar of graphics approach is that users can compose plots by explicitly mapping data to the various elements that make up the plot. It is a particularly effective approach for a whole slew of standard plots created from tidy data.

Before playing with **plotnine** though, I found that my preferred settings for **maplotlib** made the (very sensible) defaults for **plotnine** go awry, particularly for facet plots, so I'm going to switch back to the matplotlib defaults:


In [ ]:
import matplotlib as mpl

mpl.rcParams.update(mpl.rcParamsDefault)

Let's take a look at how to do a simple scatter plot, like we saw for **matplotlib**, in **plotnine**. We'll use the *mtcars* dataset.

In [ ]:
from plotnine import ggplot, geom_point, aes
from plotnine.data import mtcars

(ggplot(mtcars, aes("wt", "mpg")) + geom_point())

Here, `ggplot` is the organising framework for creating a plot and `mtcars` is a dataframe with the data in that we'd like to plot. `aes` stands for aesthetic mapping and it tells **plotnine** which columns of the dataframe to treat as the x and y axis (in that order). Finally, `geom_point` tells **plotnine** to add scatter points to the plot.

If we want to add colour, we pass a colour keyword argument to `aes` like so (with 'factor' meaning treat the variable like it's a categorical):


In [ ]:
(ggplot(mtcars, aes("wt", "mpg", color="factor(gear)")) + geom_point())

One of the nice aspects of the grammar of graphics approach, perhaps its best feature, is that switching to other types of 'geom' (aka chart type) is as easy as calling the same code but with a different 'geom' switched in. Note that, because we only imported one element at a time from **plotnine** we do need to explicitly import any other 'geoms' that we'd like to use, as in the next example below. But we *could* have just imported everything from **plotnine** instead using `from plotnine import *`.

The next example shows how easy it is to switch between 'geoms'.


In [ ]:
from plotnine import geom_smooth

(ggplot(mtcars, aes("wt", "mpg")) + geom_smooth())

Furthermore, we can add *multiple* geoms to the same chart by layering them within the same call to the `ggplot` function:

In [ ]:
(ggplot(mtcars, aes("wt", "mpg")) + geom_smooth(color="blue") + geom_point())

Just like **seaborn** and **matplotlib**, we can create facet plots too--but this time they're just a variation on the same underlying call to `ggplot`. Let's see that same example of GDP by country rendered with **plotnine**: 

In [ ]:
from plotnine import geom_line, facet_wrap, theme, element_text

(
    ggplot(tidy_df, aes(x="Date", y="Real GDP growth, %", color="factor(Region)"))
    + geom_line()
    + facet_wrap("Country", nrow=2)
    + theme(axis_text_x=element_text(rotation=90))
)

**plotnine** can do many of the same types of charts as **seaborn**; let's see some similar examples:

In [ ]:
from plotnine import geom_violin, scale_fill_manual

(
    ggplot(tips, aes("day", "total_bill", fill="smoker"))
    + geom_violin(tips)
    + scale_fill_manual(values=["dodgerblue", "darkorange"])
)

In [ ]:
from plotnine import labs

(
    ggplot(df, aes(x="bill_length_mm", y="bill_depth_mm", color="factor(species)"))
    + geom_point()
    + geom_smooth(method="lm")
    + labs(x="Bill length (mm)", y="Bill depth (mm)")
)

Finally, an example of great practical use during exploratory analysis, the kernel density plot:

In [ ]:
from plotnine import geom_density
from plotnine.data import mpg

(ggplot(mpg, aes(x="cty", color="drv", fill="drv")) + geom_density(alpha=0.1))

### Altair

[**Altair**](https://altair-viz.github.io/) is similar to **plotnine** and **seaborn**, in that it's declarative, but it's very dissimilar in its look and feel. It isn't based on **matplotlib** and it's really more suited to displaying graphics on the web than anything else. Under the hood, it uses powerful javascript visualisation libraries. Because you're unlikely to use it in a paper, we won't give it quite the same amount of time or consideration as the other libraries.

A note of caution about **altair**: you shouldn't use it for very large datasets (see the guidance [here](https://altair-viz.github.io/user_guide/faq.html#altair-faq-large-notebook)--there is a way around it) and, last time I tried, charts couldn't easily be exported to a PDF friendly format. It's really geared around web visualisation, and very beautiful web vis at that!

The first thing to understand about **altair** is that, in a similar way to **plotnine**, the creation of a chart is always the same and it's only the specification of the type of chart that happens differently. Here's a by now familiar scatterplot example:

In [ ]:
import altair as alt
from vega_datasets import data

cars = data.cars()

alt.Chart(cars).mark_circle(size=60).encode(
    x="Horsepower", y="Miles_per_Gallon", color="Origin", tooltip=["Name"]
).interactive()

There's a lot to unpack here! The very basic elements of the chart come from calling

```python
alt.Chart(cars).mark_circle().encode(
    x='Horsepower',
    y='Miles_per_Gallon',
    color='Origin',
)
```

as this asks for a chart using the (tidy) mtcars dataframe with circles that are coloured by the variable 'gear' treated as an ordinal (aka a category or factor) using `color=gear:O`. I had to specify that this should be  There are some extras. I added bigger circles and interactivity so that you can i) move the chart around as you like and ii) if you hover over any of the points, you will see the 'name' variable for that entry displayed. The latter is achieved via the tooltip keyword argument.

As with ggplot, we can add multiple elements to a single plot. In this case, **altair** uses a clever property of Python called overloading to 'overload' the addition operator so that chart1 + chart2 creates a combined chart of both:

In [ ]:
points = (
    alt.Chart(cars)
    .mark_point()
    .encode(x="Horsepower", y="Miles_per_Gallon", tooltip=["Name"])
)

line = points.transform_loess("Horsepower", "Miles_per_Gallon").mark_line(color="black")

(points + line).interactive()

They have also overloaded other symbols so that charts can be shown in relative positions:

In [ ]:
points = (
    alt.Chart(cars)
    .mark_circle(size=60)
    .encode(x="Horsepower", y="Miles_per_Gallon", color="Origin", tooltip=["Name"])
)

hist = (
    alt.Chart(cars)
    .mark_bar()
    .encode(
        x="count()",
        y=alt.Y("Miles_per_Gallon:Q", bin=alt.Bin(maxbins=30)),
        color="Origin:N",
    )
)

points | hist

Note that `:N` tells **altair** to treat the variable preceeding the colon as a nominal value, ditto `:Q` for quantitative values. Most of the time **altair** will guess what you need, but it's easy to specify it if you need to. More complex combinations of chart arrangements may be achieved with the `alt.hconcat` and `alt.vconcat` functions.

Let's now return to some of our favourite examples and see how they look in **altair**.

In [ ]:
alt.Chart(tidy_df).mark_line().encode(
    x="Date:T",
    y="Real GDP growth, %",
    color="Region:N",
    facet=alt.Facet("Country:N", columns=2),
)

In [ ]:
alt.Chart(mpg).transform_density(
    density="cty", as_=["cty", "density"], groupby=["drv"]
).mark_area(fillOpacity=0.8).encode(
    x="cty:Q",
    y="density:Q",
    color="drv:N",
)

In [ ]:
chart = (
    alt.Chart(df)
    .mark_point()
    .encode(x="bill_length_mm", y="bill_depth_mm", color="species:N")
)

(
    chart
    + chart.transform_regression("bill_length_mm", "bill_depth_mm", groupby=["species"])
    .mark_line()
    .interactive()
)

In [ ]:
alt.Chart(tips).transform_density(
    "total_bill", as_=["total_bill", "density"], groupby=["day"]
).mark_area(orient="horizontal").encode(
    y="total_bill:Q",
    color="day:N",
    x=alt.X(
        "density:Q",
        stack="center",
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0], grid=False, ticks=True),
    ),
    column=alt.Column(
        "day:N",
        header=alt.Header(
            titleOrient="bottom",
            labelOrient="bottom",
            labelPadding=0,
        ),
    ),
).properties(
    width=100
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

I'm not an expert at **altair**, and there are many more (and, frankly, better) examples in their [gallery](https://altair-viz.github.io/gallery/index.html). It's a really rich tool though and, in expert hands, could be used to create the kind of visualisations that you might see in newspaper.

But let's just see what someone who *does* know what they're doing can achieve using it using one of their examples (and how poorly South London is served by tube lines!):

In [ ]:
boroughs = alt.topo_feature(data.londonBoroughs.url, "boroughs")
tubelines = alt.topo_feature(data.londonTubeLines.url, "line")
centroids = data.londonCentroids.url

background = (
    alt.Chart(boroughs)
    .mark_geoshape(stroke="white", strokeWidth=2)
    .encode(
        color=alt.value("#eee"),
    )
    .properties(width=700, height=500)
)

labels = (
    alt.Chart(centroids)
    .mark_text()
    .encode(
        longitude="cx:Q",
        latitude="cy:Q",
        text="bLabel:N",
        size=alt.value(8),
        opacity=alt.value(0.6),
    )
    .transform_calculate(
        "bLabel",
        "indexof (datum.name,' ') > 0  ? substring(datum.name,0,indexof(datum.name, ' ')) : datum.name",
    )
)

line_scale = alt.Scale(
    domain=[
        "Bakerloo",
        "Central",
        "Circle",
        "District",
        "DLR",
        "Hammersmith & City",
        "Jubilee",
        "Metropolitan",
        "Northern",
        "Piccadilly",
        "Victoria",
        "Waterloo & City",
    ],
    range=[
        "rgb(137,78,36)",
        "rgb(220,36,30)",
        "rgb(255,206,0)",
        "rgb(1,114,41)",
        "rgb(0,175,173)",
        "rgb(215,153,175)",
        "rgb(106,114,120)",
        "rgb(114,17,84)",
        "rgb(0,0,0)",
        "rgb(0,24,168)",
        "rgb(0,160,226)",
        "rgb(106,187,170)",
    ],
)

lines = (
    alt.Chart(tubelines)
    .mark_geoshape(filled=False, strokeWidth=2)
    .encode(
        alt.Color(
            "id:N", legend=alt.Legend(title=None, orient="bottom-right", offset=0)
        )
    )
)

background + labels + lines

### Other Data Visualisation Tools

There are tons of data visualisation libraries in Python, so many that most cannot be featured in great detail. Here are a few more that may be worth looking into depending on what you need.

Here's a quick run down of the other libraries that are available:

- [**proplot**](https://proplot.readthedocs.io/en/latest/index.html) aims to be "A lightweight matplotlib wrapper for making beautiful, publication-quality graphics", though the style is more similar to how people might make plots in the hard sciences rather than the social sciences. The point of this library is to take some of the verbosity out of **matplotlib**.
- if you're using very big data in machine learning models, it might be worth looking at Facebook's [**hiplot**](https://github.com/facebookresearch/hiplot)
- [Seaborn image](https://seaborn-image.readthedocs.io/en/latest/) does for image data what **seaborn** does for numerical and categorical data
- [Lit](https://pair-code.github.io/lit/) provides an open-source platform for visualization and understanding of NLP models (very impressive)
- [Wordcloud](https://github.com/amueller/word_cloud) does exactly what you'd expect (but use word clouds *very* sparingly!)
- [VisPy](http://vispy.org) for very large datasets plotted with WebGL and GPU acceleration.
- [PyQtGraph](http://www.pyqtgraph.org/), a pure-Python graphics library for PyQt5/PySide2 and intended for use in (heavy) mathematics / scientific / engineering applications (not very user friendly).
- [bokeh](https://docs.bokeh.org/en/latest/index.html#) offers interactive web plotting in Python.
- [HoloViews](https://holoviews.org/), a library dsigned to make data analysis and visualization seamless and simple with very concise commands (builds on bokeh and matplotlib).
- [YellowBrick](https://www.scikit-yb.org/en/latest/) for visualisations of machine learning models and metrics.
- [facets](https://pair-code.github.io/facets/) for quickly visualising machine learning features (aka regressors). Also useful for exploratory data analysis.
- [chartify](https://github.com/spotify/chartify), Spotify's quick plotting library for data scientists.
- [scikit-plot](https://github.com/reiinakano/scikit-plot) offers plotting tools designed around Python's wildy popular scikit-learn machine learning library.
- [themepy](https://github.com/petermckeeverPerform/themepy) is an open source theme selector / creator and aesthetic manager for Matplotlib.
- [scienceplots](https://github.com/garrettj403/SciencePlots) provides scientific plotting styles--some associated with specific journals--for Matplotlib.
- [colour](https://www.colour-science.org/) provides professional level colour tools for Python.
- [palettable](https://jiffyclub.github.io/palettable/) has extra colour palettes that work well with Matplotlib.
- [colorcet](https://colorcet.holoviz.org/) is a collection of perceptually uniform colourmaps.
- [missingno](https://github.com/ResidentMario/missingno) for visualization of missing data.
- [bashplotlib](https://github.com/glamp/bashplotlib), for when you want to make visualisations directly from the command line (I don't imagine this will be very often, but always good to know the option is there!)

You can see an overview of all Python plotting tools at [PyViz](https://pyviz.org/tools.html).

## Review

If you know:

- ✅ a little bit about how to use data visualisation; and
- ✅ what some of the most popular libraries for data vis are in Python

then you are well on your way to being a whizz with data vis!